# U-Net 3D V100 Analysis 

## Notebook Overview

This notebook presents an analysis of a U-Net 3D workload using the DFAnalyzer tool. It demonstrates how to analyze I/O traces collected from a deep learning application running on a V100 system. The workflow includes:

- Setting up the environment and importing necessary libraries.
- Extracting and preparing trace data for analysis.
- Initializing DFAnalyzer with appropriate configuration.
- Running the analysis to generate summarized I/O statistics and views.
- Displaying and interpreting the results, including bandwidth and operation counts over time ranges for different I/O layers.

The notebook is intended to help users understand the I/O behavior of deep learning workloads and provides a template for similar analyses on other datasets.

## Interactive Analysis

### Prepare Environment

In this section, we set up the environment by importing required libraries, configuring warning filters, and updating the Python path to include the DFAnalyzer workspace. 

In [1]:
import os
import sys

# Add DFAnalyzer to the path
workspace_dir = os.path.abspath("../")
sys.path.append(workspace_dir)

### Prepare Trace Data

Then, we extract the trace data archive into the designated directory to prepare it for analysis with DFAnalyzer.

In [4]:
!mkdir -p {workspace_dir}/tests/data/extracted/dftracer-dlio
!tar -xzf {workspace_dir}/tests/data/dftracer-dlio.tar.gz -C {workspace_dir}/tests/data/extracted/dftracer-dlio

### Run Analysis

Finaly, we initialize the DFAnalyzer with the specified configuration and run the trace analysis to generate summarized I/O statistics and views for further exploration.

In [ ]:
from dftracer.analyzer import init_with_hydra

percentile = 0.9
time_granularity = 5  # 5 seconds
trace_path = f"{workspace_dir}/tests/data/extracted/dftracer-dlio"
view_types = ["time_range", "proc_name"]

dfa = init_with_hydra(
    hydra_overrides=[
        'analyzer=dftracer',
        'analyzer/preset=dlio',
        'analyzer.checkpoint=False',
        f"analyzer.time_granularity={time_granularity}",
        f"trace_path={trace_path}",
    ]
)

We access the underlying Dask client via our Python API.

In [3]:
dfa.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 96,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42305,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 96
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:36801,Total threads: 8
Dashboard: http://127.0.0.1:39925/status,Memory: 0 B
Nanny: tcp://127.0.0.1:39455,


We access to current preset configuration as follows.

In [4]:
dict(dfa.analyzer.preset.layer_defs)

{'app': 'func_name == "DLIOBenchmark.run"',
 'training': 'func_name == "DLIOBenchmark._train"',
 'compute': 'cat == "ai_framework"',
 'fetch_data': 'func_name.isin(["<module>.iter", "fetch-data.iter", "loop.iter"])',
 'data_loader': 'cat == "data_loader" & ~func_name.isin(["loop.iter", "loop.yield"])',
 'data_loader_fork': 'cat == "posix" & func_name == "fork"',
 'reader': 'cat == "reader"',
 'reader_posix_lustre': 'cat.str.contains("posix|stdio") & cat.str.contains("_reader_lustre")',
 'checkpoint': 'cat == "checkpoint"',
 'checkpoint_posix_lustre': 'cat.str.contains("posix|stdio") & cat.str.contains("_checkpoint_lustre")',
 'checkpoint_posix_ssd': 'cat.str.contains("posix|stdio") & cat.str.contains("_checkpoint_ssd")',
 'other_posix': 'cat.isin(["posix", "stdio"])'}

We run the analysis via the `analyze_trace` function as follows.

In [5]:
result = dfa.analyze_trace(view_types=view_types)

And, using the `output` variable available in our analyzer instance `dfa`, we output the DFAnalyzer summary.

In [13]:
dfa.output.handle_result(result)

                                                  Process Summary                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Metric                                                                     ┃ Unit          ┃              Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ Job Time                                                                   │ seconds       │             56.695 │
│ Total Count                                                                │ count         │             15,901 │
│ Total Files                                                                │ count         │                 26 │
│ Total Nodes                                                                │ count         │                  0 │
│ Total Processes                                                            │ count         │                 56 │
│ App Count                                                                  │ count         │                  8 │
│ Training Count                                                             │ count         │                 40 │
│ Compute Count                                                              │ count         │                200 │
│ Fetch Data Count                                                           │ count         │                160 │
│ Data Loader Count                                                          │ count         │                808 │
│ Data Loader Fork Count                                                     │ count         │                 96 │
│ Reader Count                                                               │ count         │              4,008 │
│ Reader POSIX (Lustre) Count                                                │ count         │             10,432 │
│ Reader POSIX (Lustre) Size                                                 │ MB            │         111833.161 │
│ Reader POSIX (Lustre) Bandwidth                                            │ MB/s          │           6829.234 │
│ Reader POSIX (Lustre) Avg Transfer Size                                    │ MB            │             10.720 │
│ Checkpoint Count                                                           │ count         │                  8 │
│ Checkpoint POSIX (Lustre) Count                                            │ count         │                 45 │
│ Checkpoint POSIX (Lustre) Size                                             │ MB            │              0.011 │
│ Checkpoint POSIX (Lustre) Bandwidth                                        │ MB/s          │              3.111 │
│ Checkpoint POSIX (Lustre) Avg Transfer Size                                │ MB            │              0.000 │
│ Other POSIX Count                                                          │ count         │                 96 │
└────────────────────────────────────────────────────────────────────────────┴───────────────┴────────────────────┘
                                          Layer Breakdown (w/ overlap %)                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ Layer                      ┃       Time (s) ┃            Ops ┃   Ops/sec ┃         Size (MB) ┃ Bandwidth (MB/s) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ App                        │  55.246 (----) │       8 (----) │     0.145 │                 - │                - │
│ Training                   │  54.936 (----) │      40 (----) │     0.728 │                 - │                - │
│ Compute                    │  34.045 (----) │     200 (----) │     5.875 │                 - │                - │
│ Fetch Data                 │  16.361 (100%) │     160 

### Result Exploration

We access the high-level characteristics and layer-based characteristics and metrics via our Python API as follows:

View aggregated metrics across all layers, grouped by time intervals:

In [14]:
result.get_flat_view('time_range').head(10)

,app_count_max,app_count_mean,app_count_min,app_count_per,app_count_std,app_count_sum,app_ops_max,app_ops_mean,app_ops_min,app_ops_pct,...,u_reader_posix_lustre_other_time_max,u_reader_posix_lustre_read_time_max,u_reader_posix_lustre_seek_time_max,u_reader_posix_lustre_stat_time_max,u_reader_posix_lustre_sync_time_max,u_reader_posix_lustre_time_max,u_reader_posix_lustre_write_time_max,u_reader_preprocess_time_max,u_reader_sample_time_max,u_reader_time_max
time_range,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,0.0,8.0,0.018101,0.018101,0.018101,1.0,...,<NA>,0.271525,<NA>,0.0,<NA>,0.289335,<NA>,0.0,0.345637,2.086666
1,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.445384,<NA>,0.0,<NA>,0.463426,<NA>,0.0,0.607644,3.992251
2,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.0,<NA>,0.0,<NA>,0.0,<NA>,0.0,0.0,1.930469
3,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.0,<NA>,0.0,<NA>,0.0,<NA>,0.0,0.0,1.525373
4,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.0,<NA>,0.0,<NA>,0.0,<NA>,0.0,0.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.0,<NA>,0.0,<NA>,0.0,<NA>,0.0,0.0,0.308971
6,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.0,<NA>,0.0,<NA>,0.0,<NA>,0.0,0.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.0,<NA>,0.0,<NA>,0.0,<NA>,0.0,0.0,0.0
8,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,...,<NA>,0.0,<NA>,0.0,<NA>,0.0,<NA>,0.0,0.0,0.0


List all the layers available for detailed analysis:

In [15]:
result.layers

['app',
 'training',
 'compute',
 'fetch_data',
 'data_loader',
 'data_loader_fork',
 'reader',
 'reader_posix_lustre',
 'checkpoint',
 'checkpoint_posix_lustre',
 'checkpoint_posix_ssd',
 'other_posix']

Show the high-level metrics for the `app` layer:

In [16]:
result.get_hlm('app').head()

time  \
time_range proc_name                     cat            func_name         acc_pat io_cat              
0          app#corona171#1028571#1028571 dlio_benchmark DLIOBenchmark.run 0       6       55.245851   
           app#corona171#1028567#1028567 dlio_benchmark DLIOBenchmark.run 0       6       55.245983   
           app#corona171#1028568#1028568 dlio_benchmark DLIOBenchmark.run 0       6       55.245884   
           app#corona171#1028573#1028573 dlio_benchmark DLIOBenchmark.run 0       6       55.245845   
           app#corona171#1028574#1028574 dlio_benchmark DLIOBenchmark.run 0       6       55.245876   

                                                                                          count  \
time_range proc_name                     cat            func_name         acc_pat io_cat          
0          app#corona171#1028571#1028571 dlio_benchmark DLIOBenchmark.run 0       6           1   
           app#corona171#1028567#1028567 dlio_benchmark DLIOBenchmark.run 0       6           1   
           app#corona171#1028568#1028568 dlio_benchmark DLIOBenchmark.run 0       6           1   
           app#corona171#1028573#1028573 dlio_benchmark DLIOBenchmark.run 0       6           1   
           app#corona171#1028574#1028574 dlio_benchmark DLIOBenchmark.run 0       6           1   

                                                                                          size  \
time_range proc_name                     cat            func_name         acc_pat io_cat         
0          app#corona171#1028571#1028571 dlio_benchmark DLIOBenchmark.run 0       6       <NA>   
           app#corona171#1028567#1028567 dlio_benchmark DLIOBenchmark.run 0       6       <NA>   
           app#corona171#1028568#1028568 dlio_benchmark DLIOBenchmark.run 0       6       <NA>   
           app#corona171#1028573#1028573 dlio_benchmark DLIOBenchmark.run 0       6       <NA>   
           app#corona171#1028574#1028574 dlio_benchmark DLIOBenchmark.run 0       6       <NA>   

                                                                                          size_bin_0_4kib  \
time_range proc_name                     cat            func_name         acc_pat io_cat                    
0          app#corona171#1028571#1028571 dlio_benchmark DLIOBenchmark.run 0       6                  <NA>   
           app#corona171#1028567#1028567 dlio_benchmark DLIOBenchmark.run 0       6                  <NA>   
           app#corona171#1028568#1028568 dlio_benchmark DLIOBenchmark.run 0       6                  <NA>   
           app#corona171#1028573#1028573 dlio_benchmark DLIOBenchmark.run 0       6                  <NA>   
           app#corona171#1028574#1028574 dlio_benchmark DLIOBenchmark.run 0       6                  <NA>   

                                                                                          size_bin_4kib_16kib  \
time_range proc_name                     cat            func_name         acc_pat io_cat                        
0          app#corona171#1028571#1028571 dlio_benchmark DLIOBenchmark.run 0       6                      <NA>   
           app#corona171#1028567#1028567 dlio_benchmark DLIOBenchmark.run 0       6                      <NA>   
           app#corona171#1028568#1028568 dlio_benchmark DLIOBenchmark.run 0       6                      <NA>   
           app#corona171#1028573#1028573 dlio_benchmark DLIOBenchmark.run 0       6                      <NA>   
           app#corona171#1028574#1028574 dlio_benchmark DLIOBenchmark.run 0       6                      <NA>   

                                                                                          size_bin_16kib_64kib  \
time_range proc_name                     cat            func_name         acc_pat io_cat                         
0          app#corona171#1028571#1028571 dlio_benchmark DLIOBenchmark.run 0       6                       <NA>   
           app#corona171#1028567#1028567 dlio_benchmark DLIOBenchmark.run 0       6           

Display a layered main view of the `reader_posix_lustre` layer:

In [17]:
result.get_main_view('reader_posix_lustre').head()

bw  close_count  \
proc_name                     time_range                              
app#corona171#1028571#1028571 0           3.763764e+08            4   
                              1           3.989410e+08            7   
                              2           4.031049e+08            6   
                              3           8.000054e+08            9   
                              4           1.117199e+09           10   

                                                                            close_file_name  \
proc_name                     time_range                                                      
app#corona171#1028571#1028571 0           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              1           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              2           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              3           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              4           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   

                                            close_ops  close_time  count  \
proc_name                     time_range                                   
app#corona171#1028571#1028571 0           1225.114855    0.003265     56   
                              1           1573.741007    0.004448    103   
                              2           1433.006926    0.004187     78   
                              3           1363.429783    0.006601    117   
                              4           1370.614035    0.007296    118   

                                                    data_bw  data_count  \
proc_name                     time_range                                  
app#corona171#1028571#1028571 0            380482088.081022          36   
                              1            401318163.866653          72   
                              2            405925022.015696          54   
                              3            807332890.471577          81   
                              4           1129234851.377007          81   

                                                                             data_file_name  \
proc_name                     time_range                                                      
app#corona171#1028571#1028571 0           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              1           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              2           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              3           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   
                              4           {/p/lustre3/izzet/dlio-benchmark-test/unet3d_v...   

                                          data_intensity  ...  \
proc_name                     time_range                  ...   
app#corona171#1028571#1028571 0                      0.0  ...   
                              1                      0.0  ...   
                              2                      0.0  ...   
                              3                      0.0  ...   
                              4                      0.0  ...   

                                          write_size_bin_1gib_4gib  \
proc_name                     time_range                             
app#corona171#1028571#1028571 0                               <NA>   
                              1                               <NA>   
                              2                               <NA>   
                              3                               <NA>   
                              4                               <NA>   

                                          write_size_bin_1mib_4mib  \
proc_name                     time_range                             
app#corona171#1028571#1028571 0                               <NA>   
      

Access a specific view for `reader_posix_lustre`, grouped by time range:

In [18]:
result.get_layer_view('reader_posix_lustre', 'time_range').head()

,bw_max,bw_mean,bw_min,bw_std,bw_sum,close_count_max,close_count_mean,close_count_min,close_count_per,close_count_std,...,file_name_nunique,metadata_file_name_nunique,open_file_name_nunique,other_file_name_nunique,proc_name_nunique,read_file_name_nunique,seek_file_name_nunique,stat_file_name_nunique,sync_file_name_nunique,write_file_name_nunique
time_range,,,,,,,,,,,,,,,,,,,,,
0,3.928289e+08,3.704618e+08,3.551031e+08,1.285393e+07,2.963695e+09,4,4.0,4,0.04,0.0,...,35,35,32,0,8,32,0,33,0,0
1,4.240967e+08,3.914123e+08,3.679428e+08,1.718859e+07,3.131298e+09,8,7.25,7,0.0725,0.46291,...,64,64,64,0,8,64,0,64,0,0
2,4.452952e+08,4.189896e+08,4.031049e+08,1.577668e+07,3.351917e+09,6,5.75,5,0.0575,0.46291,...,54,54,48,0,8,48,0,48,0,0
3,9.010374e+08,8.151264e+08,7.810641e+08,3.781414e+07,6.521011e+09,10,9.25,9,0.0925,0.46291,...,82,82,74,0,8,74,0,74,0,0
4,1.200353e+09,1.133135e+09,1.018028e+09,6.035038e+07,9.065083e+09,10,9.75,9,0.0975,0.46291,...,78,78,70,0,8,70,0,70,0,0


Display the raw trace data, showing individual I/O events:

In [19]:
result._traces.head()

,func_name,cat,type,pid,tid,time_start,time_end,time,tinterval,time_range,...,size_bin_16kib_64kib,size_bin_64kib_256kib,size_bin_256kib_1mib,size_bin_1mib_4mib,size_bin_4mib_16mib,size_bin_16mib_64mib,size_bin_64mib_256mib,size_bin_256mib_1gib,size_bin_1gib_4gib,size_bin_4gib_plus
4,start,dftracer,0,1028571,1028571,0,0,0.0,<NA>,0,...,0,0,0,0,0,0,0,0,0,0
6,FileStorage.get_uri,storage,0,1028571,1028571,1300840,1300851,0.000011,<NA>,0,...,0,0,0,0,0,0,0,0,0,0
8,opendir,posix_reader_lustre,0,1028571,1028571,1300907,1304903,0.003996,<NA>,0,...,0,0,0,0,0,0,0,0,0,0
9,FileStorage.walk_node,storage,0,1028571,1028571,1300805,1305420,0.004615,<NA>,0,...,0,0,0,0,0,0,0,0,0,0
10,FileStorage.get_uri,storage,0,1028571,1028571,1305523,1305531,0.000008,<NA>,0,...,0,0,0,0,0,0,0,0,0,0
